In [1]:
# import ROOT
import uproot
import json
import os
from skhep.math.vectors import LorentzVector
import math
import numpy as np
import pandas as pd

# Create a ROOT dataframe for each dataset
# Note that we load the filenames from an external json file.
path = "root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22"
path = "/home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22"
files = json.load(open(os.path.join(os.environ["ROOTSYS"], "tutorials/dataframe", "df106_HiggsToFourLeptons.json")))
processes = files.keys()
df = {}
xsecs = {}
sumws = {}
samples = []
lumi = 10064.0

columns = ['trigE', 'trigM', 
           'lep_eta', 'lep_pt', 'lep_ptcone30', 'lep_etcone20', 
           'lep_type', 'lep_charge', 'lep_phi', 'lep_E', 
           'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'lep_z0', 'mcWeight',
           'scaleFactor_ELE', 'scaleFactor_MUON', 'scaleFactor_LepTRIGGER', 'scaleFactor_PILEUP',
          ]


def cut_trigE_trigM(trigE, trigM):
    return np.logical_not(np.logical_or(trigE, trigM))

# https://github.com/scikit-hep/scikit-hep/blob/master/skhep/math/vectors.py#L700
def good_electrons_and_muons(pdg_id, pt, eta, phi, e, trackd0pv, tracksigd0pv, z0):
    for i in range(0, len(pdg_id)):
        p = LorentzVector()
        p.setptetaphie(pt[i] / 1000.0, eta[i], phi[i], e[i] / 1000.0)
        if pdg_id[i] == 11:
            if (pt[i] < 7000 or abs(eta[i]) > 2.47 or abs(trackd0pv[i] / tracksigd0pv[i]) > 5 or abs(z0[i] * math.sin(p.theta())) > 0.5):
                return False
            else:
                if (abs(trackd0pv[i] / tracksigd0pv[i]) > 5 or abs(z0[i] * math.sin(p.theta())) > 0.5):
                    return False
    return True

# https://stackoverflow.com/questions/40045632/adding-a-column-in-pandas-df-using-a-function
def good_lep(lep_eta, lep_pt, lep_ptcone30, lep_etcone20):
    a = np.logical_and(abs(lep_eta) < 2.5, lep_pt > 5000)
    b = np.logical_and(lep_ptcone30 / lep_pt < 0.3, lep_etcone20 / lep_pt < 0.3)
    return np.logical_and(a, b)

def invariant_mass(pt, eta, phi, E):
    leptons = []
    for i in range(0, 4):
        p = LorentzVector()
        p.setptetaphie(pt[i], eta[i], phi[i], E[i])
        leptons.append(p)
    
    return (leptons[0] + leptons[1] + leptons[2] + leptons[3]).mag / 1000.0;

for p in processes:
    for d in files[p]:
        # Construct the dataframes
        folder = d[0] # Folder name
        sample = d[1] # Sample name
        xsecs[sample] = d[2] # Cross-section
        sumws[sample] = d[3] # Sum of weights
        samples.append(sample)
        filepath = "{}/4lep/{}/{}.4lep.root".format(path, folder, sample)
        print(sample, filepath)
        tree = uproot.open(filepath)['mini']
        for batch in tree.iterate(columns, outputtype=pd.DataFrame):
            a = batch
            print(a.shape)

            fail = a[np.vectorize(cut_trigE_trigM)(a.trigE, a.trigM)].index
            a.drop(fail, inplace=True)
            print(a.shape)

            a['good_lep'] = a.apply(lambda x: good_lep(x.lep_eta, x.lep_pt, x.lep_ptcone30, x.lep_etcone20), axis=1)

            print(a.shape)
            a = a[a.apply(lambda x: x.good_lep.sum() == 4, axis=1)]
            print(a.shape)

            a = a[a.apply(lambda x: x.lep_charge[x.good_lep].sum() == 0, axis=1)]
            print(a.shape)

            a['goodlep_sumtypes'] = a.apply(lambda x: x.lep_type[x.good_lep].sum(), axis=1)
            print(a.shape)

            a = a[(a.goodlep_sumtypes == 44) | (a.goodlep_sumtypes == 52) | (a.goodlep_sumtypes == 48)]
            print(a.shape)

            a = a[a.apply(lambda x: good_electrons_and_muons(x.lep_type[x.good_lep], x.lep_pt[x.good_lep], x.lep_eta[x.good_lep], x.lep_phi[x.good_lep], x.lep_E[x.good_lep], x.lep_trackd0pvunbiased[x.good_lep], x.lep_tracksigd0pvunbiased[x.good_lep], x.lep_z0[x.good_lep]), axis=1)]
            print(a.shape)

            a['goodlep_pt'] = a.apply(lambda x: x.lep_pt[x.good_lep], axis=1)
            a['goodlep_eta'] = a.apply(lambda x: x.lep_eta[x.good_lep], axis=1)
            a['goodlep_phi'] = a.apply(lambda x: x.lep_phi[x.good_lep], axis=1)
            a['goodlep_E'] = a.apply(lambda x: x.lep_E[x.good_lep], axis=1)
            print(a.shape)

            a = a[a.apply(lambda x: x.goodlep_pt[0] > 25000 and x.goodlep_pt[1] > 15000 and x.goodlep_pt[2] > 10000, axis=1)]
            print (a.shape)
            
            a['m4l'] = a.apply(lambda x: invariant_mass(x.goodlep_pt, x.goodlep_eta, x.goodlep_phi, x.goodlep_E), axis=1)
            
            if "data" in sample.lower():
                a['weight'] = a.apply(lambda x: 1.0, axis=1)
            else:
                a['weight'] = a.apply(lambda x: x.scaleFactor_ELE * x.scaleFactor_MUON * x.scaleFactor_LepTRIGGER * x.scaleFactor_PILEUP * x.mcWeight * xsecs[sample] / sumws[sample] * lumi, axis=1)
            print (a.shape)
            
            if sample not in df:
                df[sample] = a
                
            else:
                df[sample].append(a)
        

# peek at the last item built for seeing what they may look like
print(sample)
print(df[sample].keys())

data_A /home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22/4lep/Data/data_A.4lep.root
(39, 18)
(39, 18)
(39, 19)
(30, 19)
(24, 19)
(24, 20)
(21, 20)
(20, 20)
(20, 24)
(18, 24)
(18, 26)
data_B /home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22/4lep/Data/data_B.4lep.root
(156, 18)
(156, 18)
(156, 19)
(101, 19)
(83, 19)
(83, 20)
(62, 20)
(61, 20)
(61, 24)
(54, 24)
(54, 26)
data_C /home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22/4lep/Data/data_C.4lep.root
(237, 18)
(237, 18)
(237, 19)
(166, 19)
(141, 19)
(141, 20)
(117, 20)
(115, 20)
(115, 24)
(104, 24)
(104, 26)
data_D /home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22/4lep/Data/data_D.4lep.root
(400, 18)
(400, 18)
(400, 19)
(286, 19)
(236, 19)
(236, 20)
(206, 20)
(198, 20)
(198, 24)
(179, 24)
(179, 26)
mc_345060.ggH125_ZZ4lep /home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root
(104555, 18)
(104555, 18)
(104555, 19)
(93975, 19)
(9259

In [ ]:
histos = {}
for s in samples:
    histos[s] = df[s].Histo1D(ROOT.RDF.TH1DModel(s, "m4l", 24, 80, 170), "m4l", "weight")
# Run the event loop and merge histograms of the respective processes
def merge_histos(label):
    h = None
    for i, d in enumerate(files[label]):
        t = histos[d[1]].GetValue()
        if i == 0: h = t.Clone()
        else: h.Add(t)
    h.SetNameTitle(label, label)
    return h
data = merge_histos("data")
higgs = merge_histos("higgs")
zz = merge_histos("zz")
other = merge_histos("other")
# Apply MC correction for ZZ due to missing gg->ZZ process
zz.Scale(1.3)
# Create the plot
# Set styles
ROOT.gROOT.SetStyle("ATLAS")
# Create canvas with pad
c = ROOT.TCanvas("c", "", 600, 600)
pad = ROOT.TPad("upper_pad", "", 0, 0, 1, 1)
pad.SetTickx(False)
pad.SetTicky(False)
pad.Draw()
pad.cd()
# Draw stack with MC contributions
stack = ROOT.THStack()
for h, color in zip([other, zz, higgs], [(155, 152, 204), (100, 192, 232), (191, 34, 41)]):
    h.SetLineWidth(1)
    h.SetLineColor(1)
    h.SetFillColor(ROOT.TColor.GetColor(*color))
    stack.Add(h)
stack.Draw("HIST")
stack.GetXaxis().SetLabelSize(0.04)
stack.GetXaxis().SetTitleSize(0.045)
stack.GetXaxis().SetTitleOffset(1.3)
stack.GetXaxis().SetTitle("m_{4l}^{H#rightarrow ZZ} [GeV]")
stack.GetYaxis().SetTitle("Events")
stack.GetYaxis().SetLabelSize(0.04)
stack.GetYaxis().SetTitleSize(0.045)
stack.SetMaximum(33)
stack.GetYaxis().ChangeLabel(1, -1, 0)
# Draw data
data.SetMarkerStyle(20)
data.SetMarkerSize(1.2)
data.SetLineWidth(2)
data.SetLineColor(ROOT.kBlack)
data.Draw("E SAME")
# Add legend
legend = ROOT.TLegend(0.60, 0.65, 0.92, 0.92)
legend.SetTextFont(42)
legend.SetFillStyle(0)
legend.SetBorderSize(0)
legend.SetTextSize(0.04)
legend.SetTextAlign(32)
legend.AddEntry(data, "Data" ,"lep")
legend.AddEntry(higgs, "Higgs", "f")
legend.AddEntry(zz, "ZZ", "f")
legend.AddEntry(other, "Other", "f")
legend.Draw("SAME")
# Add ATLAS label
text = ROOT.TLatex()
text.SetNDC()
text.SetTextFont(72)
text.SetTextSize(0.045)
text.DrawLatex(0.21, 0.86, "ATLAS")
text.SetTextFont(42)
text.DrawLatex(0.21 + 0.16, 0.86, "Open Data")
text.SetTextSize(0.04)
text.DrawLatex(0.21, 0.80, "#sqrt{s} = 13 TeV, 10 fb^{-1}")
# Save the plot
c.SaveAs("HiggsToFourLeptons.pdf")